## Building the logistic regression algorithm from scratch

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
data = sm.datasets.fair.load_pandas().data
data['affairs'] = (data.affairs > 0).astype(int)
x = data.drop('affairs', axis = 1)
y = data['affairs']

In [2]:
x.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0


In [3]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: affairs, dtype: int32

In [13]:
class logisticRegression():
    
    
    def __init__(self):
        self.coef = 0
        
    def sigmoid(self, array):
        """
        Input:  This function will take a numpy array
        Output: This function will return an np.array of probabilities (values between 0 and 1)
        """
        
        if array.shape[0] > 0:
            try:
                sigm = 1/(1 + np.exp(-array))
            except:
                raise ValueError("Error in sigmoid function calculation")
        return sigm        
        

        
        
    def fit(self, x, y, alpha, threshold):
                          
        """
        x: input pandas dataframe
        y: output pandas series (Class labels)
 
        
        step1: Check if the number of rows in x and y are the same. If not raise a value error with a message.
        """
        if not (x.shape[0] == y.shape[0]):
            raise ValueError("The number of predictions is not euqal to number of inputs")
        
        
        """        
        step2: Check if there is any missing value in the dataset (both for x and y). 
               If there is, raise a value error with a message."""
        if x.isnull().values.any():
            raise ValueError("Input data contains missing values")
        if y.isnull().values.any():
            raise ValueError("Given Output data contains missing values")
        
        
        """         
        step3: Check if there is any categorical value in x or y. If there is, raise a value error with a message."""
        if len(x.select_dtypes(['object']).columns) > 0:
            raise ValueError("Input data contains categorical values")
        if y.dtype == 'object':
            raise ValueError("Output data is categorical")
        
        
        """        
        step4: Transform both x and y into numpy.arrays (it is easier to work with arrays for matrix operations)"""
        try:
            x = np.asarray(x)
            y = np.asarray(y)
        except:
            raise ValueError("Error converting x or y to numpy as array")
        
        
        """
        step5: Add bias to the input vector x. bias means add a column which is 1 across al the rows.
               This will increase the number of columns of x by 1. x.shape[1] will increase by 1."""
        try:
            bias = np.ones((len(x), 1))
            x = np.concatenate((bias, x), axis = 1)
            print("input values with added bias \n", x)
        except:
            raise ValueError("error adding bias values to input")
        
        
        """       
        step6: initialize self.coef."""
        
        coefficients = np.random.uniform(-1, 1, x.shape[1])      
        print("\nCoefficients\n", coefficients)
        self.coef = coefficients       
        

        
        """       
        step7: create a list to save the cost values for each iteration."""
        cost = []
        
        
        """
        step8: while not converged and iteration number > 10000
                    calculate the predicted values
                    calculate the error 
                    calculate the cost function and append it to the cost list
                    calculate the gradient in a way that gradient is 
                                      gradient = (t(x) * (error))/(size_of_x) (number of rows)
                    adjust the coef in a way that
                                        coef = coef - alpha*gradient
                    adjust alpha in a way that
                                        alpha = alpha*0.95
                    
        step 8: Check if the convergence criteria is satisfied:
                if you iterate at least as many times 10000
                if the difference between the average of the last 5 cost values and the last cost value 
                is less than the threshold."""
        
        i = 0
        isconverge = False
        while ((not isconverge) and i < 10000): 
            pred = self.predict_prob(x)
        
            error = y - pred
            print("\nerror values\n", error)
        
            try:
                newcost = np.sum((-y+0.0001) * np.log(pred+0.0001) - (1-y) * np.log(1-pred+0.0001))/x.shape[0]
                print("\nNew Cost\n", newcost)
                cost.append(newcost)

            except:
                raise ValueError("Error calculating or appending cost value")


            gradient = np.dot(x.T, error)/x.shape[0]
            print("\nGradient Values\n", gradient)

            self.coef = self.coef - alpha * gradient
            print("\nNew Coeff\n", self.coef)
            
            i = i + 1
            isconverge = ((np.average(self.coef[-5:]) - self.coef[-1]) < threshold)
            print("\nIs Model Convergent: ", isconverge)
        
        print('\nNumber of iterations: ',i)
        
        accuracy = self.get_accuracy(x,y,threshold)
        print("\nAccuracy of predications is\n",accuracy)
        
    
    def predict_prob(self, x):
        
        """
        Convert x into numpy aray and add bias
        Check if size of self.coef is the same with the number of columns in x
        Using x and self.coef, make the predictions
        """
        print("\nInside predict_prob function")
        try:
            arr = np.dot(x, self.coef)
            print('\nInput to sigmoid\n', arr)

            prob = self.sigmoid(arr)
            print("predicted probabilities\n",prob)
        except:
            raise ValueError("Error predicting probabilities")
        return prob
    
    
    def predict_class(self, x, threshold):
        """
        Make discrete predictions. Instead of returning probabilities return 0 or 1.
        """
        print("Inside predict_class function")
        try:
            arr = np.dot(x, self.coef)
            prob = self.sigmoid(arr)
            prob[prob > threshold] = 1
            prob[prob <= threshold] = 0
        except:
            raise ValueError("Error predicting class")
        return prob
    
    def get_accuracy(self, x, y, threshold):
        
        """
        Calculate the accuracy rate
        number of true classification/total number of instances
        number of true classification is True positive + True negative
        """
        pred_classes = np.array(self.predict_class(x, threshold))
        true_classifications = (pred_classes == y).sum()
        accuracy = true_classifications / y.shape[0]
        return accuracy

In [16]:
lr = logisticRegression()
lr.fit(x,y,0.05,0.5)

input values with added bias 
 [[ 1.  3. 32. ... 17.  2.  5.]
 [ 1.  3. 27. ... 14.  3.  4.]
 [ 1.  4. 22. ... 16.  3.  5.]
 ...
 [ 1.  5. 22. ... 14.  3.  1.]
 [ 1.  5. 32. ... 14.  3.  4.]
 [ 1.  4. 22. ... 16.  2.  4.]]

Coefficients
 [ 0.35260234 -0.08097451  0.1503879  -0.70062422 -0.05885119 -0.55090023
 -0.16623144  0.4124991  -0.60022953]

Inside predict_prob function

Input to sigmoid
 [-8.2148645  -9.15607745 -3.38857602 ... -1.28706978 -4.64581553
 -3.75174583]
predicted probabilities
 [2.70527970e-04 1.05565070e-04 3.26544062e-02 ... 2.16349194e-01
 9.51038024e-03 2.29382096e-02]

error values
 [ 0.99972947  0.99989443  0.96734559 ... -0.21634919 -0.00951038
 -0.02293821]

New Cost
 2.103184332103264

Gradient Values
 [0.27504233 0.97644322 8.6979918  3.49475108 0.54505626 0.6328218
 3.83811123 0.95100276 1.12303335]

New Coeff
 [ 0.33885023 -0.12979667 -0.28451169 -0.87536177 -0.08610401 -0.58254132
 -0.358137    0.36494897 -0.65638119]

Is Model Convergent:  True

Number 